# Distributed deep learning training using PyTorch with HorovodRunner for MNIST

This notebook illustrates the use of HorovodRunner for distributed training using PyTorch. 
It first shows how to train a model on a single node, and then shows how to adapt the code using HorovodRunner for distributed training. 
The notebook runs on both CPU and GPU clusters.

## Setup Requirements
Databricks Runtime 7.6 ML or above (choose either CPU or GPU clusters). Torch and its related packages and Horovod are preinstalled with this runtime.
HorovodRunner is designed to improve model training performance on clusters with multiple workers, but multiple workers are not required to run this notebook.

## Set up checkpoint location
The next cell creates a directory for saved checkpoint models. Databricks recommends saving training data under `dbfs:/ml`, which maps to `file:/dbfs/ml` on driver and worker nodes.

In [0]:
PYTORCH_DIR = '/dbfs/ml/horovod_pytorch'

## Prepare single-node code

First, create single-node PyTorch code. This is modified from the [Horovod PyTorch MNIST Example](https://github.com/horovod/horovod/blob/master/examples/pytorch/pytorch_mnist.py).

### Define a simple convolutional network

In [0]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(1, 10, kernel_size=5)
        self.conv2 = nn.Conv2d(10, 20, kernel_size=5)
        self.conv2_drop = nn.Dropout2d()
        self.fc1 = nn.Linear(320, 50)
        self.fc2 = nn.Linear(50, 10)

    def forward(self, x):
        x = F.relu(F.max_pool2d(self.conv1(x), 2))
        x = F.relu(F.max_pool2d(self.conv2_drop(self.conv2(x)), 2))
        x = x.view(-1, 320)
        x = F.relu(self.fc1(x))
        x = F.dropout(x, training=self.training)
        x = self.fc2(x)
        return F.log_softmax(x)

###Configure single-node training

In [0]:
# Specify training parameters
batch_size = 100
num_epochs = 3
momentum = 0.5
log_interval = 100

In [0]:
def train_one_epoch(model, device, data_loader, optimizer, epoch):
    model.train()
    for batch_idx, (data, target) in enumerate(data_loader):
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = F.nll_loss(output, target)
        loss.backward()
        optimizer.step()
        if batch_idx % log_interval == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * len(data), len(data_loader) * len(data),
                100. * batch_idx / len(data_loader), loss.item()))

### Create methods for saving and loading model checkpoints

In [0]:
def save_checkpoint(log_dir, model, optimizer, epoch):
  filepath = log_dir + '/checkpoint-{epoch}.pth.tar'.format(epoch=epoch)
  state = {
    'model': model.state_dict(),
    'optimizer': optimizer.state_dict(),
  }
  torch.save(state, filepath)
  
def load_checkpoint(log_dir, epoch=num_epochs):
  filepath = log_dir + '/checkpoint-{epoch}.pth.tar'.format(epoch=epoch)
  return torch.load(filepath)

def create_log_dir():
  log_dir = os.path.join(PYTORCH_DIR, str(time()), 'MNISTDemo')
  os.makedirs(log_dir)
  return log_dir

### Run single-node training with PyTorch

In [0]:
import torch.optim as optim
from torchvision import datasets, transforms
from time import time
import os

single_node_log_dir = create_log_dir()
print("Log directory:", single_node_log_dir)

def train(learning_rate):
  device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

  train_dataset = datasets.MNIST(
    'data', 
    train=True,
    download=True,
    transform=transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.1307,), (0.3081,))]))
  data_loader = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size, shuffle=True)

  model = Net().to(device)

  optimizer = optim.SGD(model.parameters(), lr=learning_rate, momentum=momentum)

  for epoch in range(1, num_epochs + 1):
    train_one_epoch(model, device, data_loader, optimizer, epoch)
    save_checkpoint(single_node_log_dir, model, optimizer, epoch)

    
def test(log_dir):
  device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
  loaded_model = Net().to(device)
  
  checkpoint = load_checkpoint(log_dir)
  loaded_model.load_state_dict(checkpoint['model'])
  loaded_model.eval()

  test_dataset = datasets.MNIST(
    'data', 
    train=False,
    download=True,
    transform=transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.1307,), (0.3081,))]))
  data_loader = torch.utils.data.DataLoader(test_dataset)

  test_loss = 0
  for data, target in data_loader:
      data, target = data.to(device), target.to(device)
      output = loaded_model(data)
      test_loss += F.nll_loss(output, target)
  
  test_loss /= len(data_loader.dataset)
  print("Average test loss: {}".format(test_loss.item()))

Log directory: /dbfs/ml/horovod_pytorch/1613601002.1851563/MNISTDemo

Run the `train` function you just created to train a model on the driver node.

In [0]:
train(learning_rate = 0.001)

Downloading http://yann.lecun.com/exdb/mnist/train-images-idx3-ubyte.gz to data/MNIST/raw/train-images-idx3-ubyte.gz
0it [00:00, ?it/s] 0%| | 0/9912422 [00:00<?, ?it/s] 0%| | 49152/9912422 [00:00<00:33, 295992.43it/s] 2%|▏ | 212992/9912422 [00:00<00:25, 382174.85it/s] 9%|▉ | 876544/9912422 [00:00<00:17, 527936.79it/s] 30%|███ | 3006464/9912422 [00:00<00:09, 743770.35it/s] 58%|█████▊ | 5709824/9912422 [00:00<00:04, 1046128.64it/s] 88%|████████▊ | 8683520/9912422 [00:01<00:00, 1449510.09it/s]9920512it [00:01, 8483849.69it/s] 
Extracting data/MNIST/raw/train-images-idx3-ubyte.gz to data/MNIST/raw
Downloading http://yann.lecun.com/exdb/mnist/train-labels-idx1-ubyte.gz to data/MNIST/raw/train-labels-idx1-ubyte.gz
0it [00:00, ?it/s] 0%| | 0/28881 [00:00<?, ?it/s]32768it [00:00, 125974.40it/s] 
Extracting data/MNIST/raw/train-labels-idx1-ubyte.gz to data/MNIST/raw
Downloading http://yann.lecun.com/exdb/mnist/t10k-images-idx3-ubyte.gz to data/MNIST/raw/t10k-images-idx3-ubyte.gz
0it [00:00, ?it/s] 0%| | 0/1648877 [00:00<?, ?it/s] 1%| | 16384/1648877 [00:00<00:09, 163305.19it/s] 6%|▌ | 98304/1648877 [00:00<00:07, 208219.52it/s] 22%|██▏ | 368640/1648877 [00:00<00:04, 284999.76it/s] 30%|██▉ | 491520/1648877 [00:00<00:03, 359526.53it/s] 44%|████▍ | 729088/1648877 [00:00<00:01, 472628.96it/s] 59%|█████▊ | 966656/1648877 [00:00<00:01, 606759.89it/s] 74%|███████▍ | 1220608/1648877 [00:01<00:00, 759183.27it/s] 89%|████████▉ | 1474560/1648877 [00:01<00:00, 956512.13it/s]1654784it [00:01, 1265048.28it/s] 
Extracting data/MNIST/raw/t10k-images-idx3-ubyte.gz to data/MNIST/raw
Downloading http://yann.lecun.com/exdb/mnist/t10k-labels-idx1-ubyte.gz to data/MNIST/raw/t10k-labels-idx1-ubyte.gz
0it [00:00, ?it/s] 0%| | 0/4542 [00:00<?, ?it/s]8192it [00:00, 73982.60it/s] 
Extracting data/MNIST/raw/t10k-labels-idx1-ubyte.gz to data/MNIST/raw
Processing...
/databricks/python/lib/python3.7/site-packages/torchvision/datasets/mnist.py:480: UserWarning: The given NumPy array is not writeable, and PyTorch does not support non-writeable tensors. This means you can write to the underlying (supposedly non-writeable) NumPy array using the tensor. You may want to copy the array to protect its data or make it writeable before converting it to a tensor. This type of warning will be suppressed for the rest of this program. (Triggered internally at /opt/conda/conda-bld/pytorch_1607370141920/work/torch/csrc/utils/tensor_numpy.cpp:141.)
 return torch.from_numpy(parsed.astype(m[2], copy=False)).view(*s)
Done!
/local_disk0/tmp/1613600927050-0/PythonShell.py:21: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.
 # running 'import PythonShell'
Train Epoch: 1 [0/60000 (0%)]	Loss: 2.373813
Train Epoch: 1 [10000/60000 (17%)]	Loss: 2.319232
Train Epoch: 1 [20000/60000 (33%)]	Loss: 2.282479
Train Epoch: 1 [30000/60000 (50%)]	Loss: 2.273975
Train Epoch: 1 [40000/60000 (67%)]	Loss: 2.267378
Train Epoch: 1 [50000/60000 (83%)]	Loss: 2.259475
Train Epoch: 2 [0/60000 (0%)]	Loss: 2.209961
Train Epoch: 2 [10000/60000 (17%)]	Loss: 2.251531
Train Epoch: 2 [20000/60000 (33%)]	Loss: 2.189429
Train Epoch: 2 [30000/60000 (50%)]	Loss: 2.155532
Train Epoch: 2 [40000/60000 (67%)]	Loss: 2.087146
Train Epoch: 2 [50000/60000 (83%)]	Loss: 2.048989
Train Epoch: 3 [0/60000 (0%)]	Loss: 1.944567
Train Epoch: 3 [10000/60000 (17%)]	Loss: 1.855910
Train Epoch: 3 [20000/60000 (33%)]	Loss: 1.668793
Train Epoch: 3 [30000/60000 (50%)]	Loss: 1.448902
Train Epoch: 3 [40000/60000 (67%)]	Loss: 1.511713
Train Epoch: 3 [50000/60000 (83%)]	Loss: 1.389288

Load and use the model

In [0]:
test(single_node_log_dir)

/local_disk0/tmp/1613600927050-0/PythonShell.py:21: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.
 # running 'import PythonShell'
Average test loss: 0.9279822111129761

## Migrate to HorovodRunner

HorovodRunner takes a Python method that contains deep learning training code with Horovod hooks. HorovodRunner pickles the method on the driver and distributes it to Spark workers. A Horovod MPI job is embedded as a Spark job using barrier execution mode.

In [0]:
import horovod.torch as hvd
from sparkdl import HorovodRunner

In [0]:
hvd_log_dir = create_log_dir()
print("Log directory:", hvd_log_dir)

def train_hvd(learning_rate):
  
  # Initialize Horovod
  hvd.init()  
  device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
  
  if device.type == 'cuda':
    # Pin GPU to local rank
    torch.cuda.set_device(hvd.local_rank())

  train_dataset = datasets.MNIST(
    # Use different root directory for each worker to avoid conflicts
    root='data-%d'% hvd.rank(),  
    train=True, 
    download=True,
    transform=transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.1307,), (0.3081,))])
  )

  from torch.utils.data.distributed import DistributedSampler
  
  # Configure the sampler so that each worker gets a distinct sample of the input dataset
  train_sampler = DistributedSampler(train_dataset, num_replicas=hvd.size(), rank=hvd.rank())
  # Use train_sampler to load a different sample of data on each worker
  train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size, sampler=train_sampler)

  model = Net().to(device)
  
  # The effective batch size in synchronous distributed training is scaled by the number of workers
  # Increase learning_rate to compensate for the increased batch size
  optimizer = optim.SGD(model.parameters(), lr=learning_rate * hvd.size(), momentum=momentum)

  # Wrap the local optimizer with hvd.DistributedOptimizer so that Horovod handles the distributed optimization
  optimizer = hvd.DistributedOptimizer(optimizer, named_parameters=model.named_parameters())
  
  # Broadcast initial parameters so all workers start with the same parameters
  hvd.broadcast_parameters(model.state_dict(), root_rank=0)

  for epoch in range(1, num_epochs + 1):
    train_one_epoch(model, device, train_loader, optimizer, epoch)
    # Save checkpoints only on worker 0 to prevent conflicts between workers
    if hvd.rank() == 0:
      save_checkpoint(hvd_log_dir, model, optimizer, epoch)

Log directory: /dbfs/ml/horovod_pytorch/1613601232.8936656/MNISTDemo

Now that you have defined a training function with Horovod,  you can use HorovodRunner to distribute the work of training the model. 

The HorovodRunner parameter `np` sets the number of processes. This example uses a cluster with two workers, each with a single GPU, so set `np=2`. (If you use `np=-1`, HorovodRunner trains using a single process on the driver node.)

In [0]:
hr = HorovodRunner(np=2) 
hr.run(train_hvd, learning_rate = 0.001)

HorovodRunner will only stream logs generated by :func:`sparkdl.horovod.log_to_driver` or
:class:`sparkdl.horovod.tensorflow.keras.LogCallback` to notebook cell output. If want to stream all
logs to driver for debugging, you can set driver_log_verbosity to 'log_callback_only', like
`HorovodRunner(np=2, driver_log_verbosity='all')`.
The global names read or written to by the pickled function are {'range', 'hvd', 'datasets', 'batch_size', 'num_epochs', 'Net', 'momentum', 'save_checkpoint', 'torch', 'transforms', 'optim', 'hvd_log_dir', 'train_one_epoch'}.
The pickled object size is 4501 bytes.

### How to enable Horovod Timeline? ###
HorovodRunner has the ability to record the timeline of its activity with Horovod Timeline. To
record a Horovod Timeline, set the `HOROVOD_TIMELINE` environment variable to the location of the
timeline file to be created. You can then open the timeline file using the chrome://tracing
facility of the Chrome browser.

Start training.

In [0]:
test(hvd_log_dir)

/local_disk0/tmp/1613600927050-0/PythonShell.py:21: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.
 # running 'import PythonShell'
Average test loss: 0.6572399735450745

Under the hood, HorovodRunner takes a Python method that contains deep learning training code with Horovod hooks. HorovodRunner pickles the method on the driver and distributes it to Spark workers. A Horovod MPI job is embedded as a Spark job using the barrier execution mode. The first executor collects the IP addresses of all task executors using BarrierTaskContext and triggers a Horovod job using `mpirun`. Each Python MPI process loads the pickled user program, deserializes it, and runs it.

For more information, see [HorovodRunner API documentation](https://databricks.github.io/spark-deep-learning/#api-documentation).